In [1]:
import pandas as pd
import openpyxl
import os
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pyparsing import col
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm 
import re

In [2]:
#검색할 식당 데이터 임포트
rest_data = pd.read_csv('종로구.csv', encoding='utf-8-sig')
rest_data

,순번,resturant,최종 지역,합친 검색단어
0,0.0,누이네,광장시장,누이네 광장시장
1,1.0,자매집,광장시장,자매집 광장시장
2,8.0,온더보더,광화문,온더보더 광화문
3,9.0,도토리브라더스,광화문,도토리브라더스 광화문
4,13.0,버거파크,대학로,버거파크 대학로
...,...,...,...,...
498,NaN,NaN,NaN,NaN
499,NaN,NaN,NaN,NaN
500,NaN,NaN,NaN,NaN
501,NaN,NaN,NaN,NaN


In [3]:
#결측치 처리 및 식당명만 변수 지정
rest_data=rest_data.dropna()
rest_data.info()

#지역명이 포함된 식당명 컬럼을 변수로 지정
items = rest_data['합친 검색단어']
print(items)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 451 entries, 0 to 450
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   순번         451 non-null    float64
 1   resturant  451 non-null    object 
 2   최종 지역      451 non-null    object 
 3   합친 검색단어    451 non-null    object 
dtypes: float64(1), object(3)
memory usage: 17.6+ KB
0         누이네 광장시장
1         자매집 광장시장
2         온더보더 광화문
3      도토리브라더스 광화문
4         버거파크 대학로
          ...     
446          빚짜 종로
447        차니오뎅 혜화
448         홍곱창 혜화
449         도도야 혜화
450        까페랑솜 혜화
Name: 합친 검색단어, Length: 451, dtype: object


In [5]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.maximize_window()


count = 0 #
current = 0 #현재 진행 상황

goal = len(items) #총 식당 수

#데이터 프레임으로 만들 빈 리스트 생성
rev_list=[]


for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    
    
    #카카오맵 접속 후 검색창에 입력
    driver.get("https://map.kakao.com/")
    
    #검색창에 식당명 입력   
    searchbox = driver.find_element(by='xpath', value="//input[@id='search.keyword.query']") 
    searchbox.send_keys(item)
    time.sleep(2)
    
  
    #검색 버튼 누르기
    searchbutton = driver.find_element(by='xpath', value="//button[@id='search.keyword.submit']") 
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    #검색한 식당이 있을 때 리뷰 정보 가져오기
    if len(driver.find_elements(by='xpath', value="//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        
        #리뷰수 버튼 클릭
        reviewbutton = driver.find_element(by='xpath', value="//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href")
        
        #리뷰창이 새 탭에서 뜨도록 처리 
        tabs = driver.window_handles
        driver.switch_to.window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(2)
        
        

        #스크롤 내리기
        for c in range(0,1):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(1)
        
        
        #리뷰 데이터 스크래핑을 위한 html 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        
        #리뷰 수가 0이 아닌 경우 리뷰글 가져오기
        if len(review_lists) != 0 :
            
            #후기 더보기가 있으면 모두 눌러주기 
            while True: 
                try:
                    if driver.execute_script('return document.querySelector("#mArticle > div.cont_evaluation > div.evaluation_review > a > span.txt_more").textContent') == "후기 더보기":
                        driver.execute_script('return document.querySelector("#mArticle > div.cont_evaluation > div.evaluation_review > a > span.ico_comm.ico_more").click()')
                        time.sleep(2) 
                        
                except:
                    print("더보기 종료")
                    break 
                
            #후기 더보기를 모두 누른 후 리뷰 리스트 재정의
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            review_lists = soup.select('.list_evaluation > li')
            print(len(review_lists))
                    
            
            #리뷰 정보가 있는 경우 리뷰글, 식당 평균 평점, 리뷰 작성 시간 가져오기     
            try:
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.comment_info > p > span') # 리뷰
                        rate = driver.find_element(by='xpath',value = '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[1]').text #식당평균평점
                        timestamp = review.select(' div > span.time_write') #시간정보
                        
                        #리뷰 정보가 있는 경우 식당 이름, 평점, 리뷰 텍스트, 작성 시간을 가져와서 데이터 프레임으로 만들기
                        rev_list.append(
                        [
                            item,
                            rate,
                            user_review[0].text,
                            timestamp[0].text
                        ]
                        )            
    
                    time.sleep(1)
                
               
            except:
                print("더 이상 리뷰가 존재하지 않습니다.")
                
                
                
        else :
            #리뷰 정보가 없는 경우 식당명과 평균 평점만 데이터 프레임에 추가
            rate = driver.find_element(by='xpath',value = '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[1]').text
            rev_list.append(
                [
                    item,
                    rate,
                ]
            ) 

                    
            print("리뷰가 없습니다")
        
        #검색한 창 닫고 카카오맵 검색 페이지로 돌아가기    
        driver.close()
        driver.switch_to.window(tabs[0])
        print("기본 페이지로 돌아가기")
            
    else:
        print("식당이 존재하지 않음")
        
driver.close() 

[WDM] - Downloading: 100%|██████████| 6.29M/6.29M [00:01<00:00, 3.71MB/s]


진행상황 : 1/2
식당 존재
더보기 종료
18
기본 페이지로 돌아가기
진행상황 : 2/2
식당 존재
더보기 종료
22
기본 페이지로 돌아가기


In [6]:
#스크래핑한 데이터를 데이터 프레임으로 만들기    
columns = ["name", "rate", "review", "timestamp"]
df = pd.DataFrame(rev_list, columns=columns)
df    

,name,rate,review,timestamp
0,로흐,4.7,완벽한 선물용.그 튀밥올라간거 어머님이 좋아하셔서 오 좀하는데 싶었어요.이게 속촉파...,2022.09.21.
1,로흐,4.7,,2022.09.08.
2,로흐,4.7,무난한듯,2022.08.29.
3,로흐,4.7,언덕에 있어서 약간 힘들지만…… 너무 맛있어요…,2022.07.21.
4,로흐,4.7,쫀득쫀득 잔짜 존맛...포장해서 얼렸다 먹으면 진짜 맛있어요!!,2022.07.14.
5,로흐,4.7,,2022.06.17.
6,로흐,4.7,겉은 바삭 속은 부드러운 휘낭시에 점심시간엔 사람이 좀 많아요오렌지 얼그레이가 의외...,2022.06.05.
7,로흐,4.7,,2022.04.26.
8,로흐,4.7,휘낭시에 대존맛 ㄹㅇ꼭가셈재방문의사 ㅇ,2022.04.25.
9,로흐,4.7,회사 근처에 방앗간이 생겨버려 큰일이에요~ ~ 얼그레이 강경파는 후회하지 않을 것.,2022.03.03.


In [7]:
#데이터 프레임 전처리#

#식당별로 리뷰가 얼마나 있는지 확인
print(df['name'].value_counts())

#중복값 제거
df = df.drop_duplicates(['review'], keep='first',ignore_index = True)

#리뷰글이 없는 결측치 제거
df['review']=df['review'].dropna()

#식당별로 리뷰가 얼마나 있는지 확인
print(df['name'].value_counts())

순도리    22
로흐     18
Name: name, dtype: int64
순도리    22
로흐     14
Name: name, dtype: int64


C:\Users\soyoung\AppData\Local\Temp\ipykernel_20428\300514302.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].dropna()


In [14]:
#스크래핑한 데이터 프레임을 csv로 저장
df.to_csv('sample.csv', encoding='utf-8-sig')